In [4]:
# MINI TEST
import pandas as pd
import os
import numpy as np
import shapely as shp
import geopandas as gpd

import pulp
import scipy as sp
from geopy.distance import great_circle
from geopy.distance import geodesic
from shapely.geometry import Point

from scipy.spatial import distance_matrix

# import plotly.plotly as py
# import plotly.graph_objs as go
# from matplotlib import rc

os.chdir("/Users/anayahall/projects/grapevine")


In [126]:
# mini gdfs of county wastes (tbm - location and MSW for 2014) 
c = gpd.read_file("data/clean/techbiomass_pts.shp")
#c = c[(c['biomass.ca'] == "organic fraction municipal solid waste") & (c['year'] == 2014)].copy()
c = c[(c['biomass.fe'] == "FOOD") & (c['year'] == 2014)].copy()

c = c[['FIPS', 'COUNTY', 'disposal.y', 'geometry']]
# subset out four counties
csub = c[(c['COUNTY'] == "Los Angeles") | (c['COUNTY'] == "San Diego")| 
         (c['COUNTY'] == "Orange")| (c['COUNTY'] == "Imperial")].copy()
# csub = c[(c['COUNTY'] == "San Diego")| (c['COUNTY'] == "Imperial")].copy()



In [97]:
# Mini gdfs of facilites (location and capacity)
f = gpd.read_file("data/clean/clean_swis.shp")
f = f[['SwisNo', 'AcceptedWa', 'County', 'cap_m3', 'geometry']].copy()

# subset out four counties
fsub = f[(f['County'] == "Los Angeles") | (f['County'] == "San Diego") | 
          (f['County'] == "Orange")| (f['County'] == "Imperial")].copy()
# too many, just select first 5
fsub = fsub[0:4].copy()
fsub

SyntaxError: invalid syntax (<ipython-input-97-de5439b6175d>, line 5)

In [127]:
def geo_to_coords(df):
    df['coord'] = ""
    for index, row in df.iterrows():
    #      print('index', index)
    #      print('row', row)
         for pt in list(row['geometry'].coords): 
    #         print(pt)
            df.at[index,'coord'] = np.asarray(pt)


In [128]:
geo_to_coords(csub)

csub

,FIPS,COUNTY,disposal.y,geometry,coord
56,6037,Los Angeles,307295.084576,POINT (-118.5720592538903 34.14803912013723),"[-118.57205925389029, 34.14803912013723]"
3206,6059,Orange,104525.755860,POINT (-117.8563081974633 33.53332689323803),"[-117.85630819746329, 33.533326893238026]"
4106,6025,Imperial,6751.167110,POINT (-115.5613145428947 32.79849442795177),"[-115.56131454289468, 32.798494427951766]"
5906,6073,San Diego,113139.598593,POINT (-117.2241057245406 32.94260371968307),"[-117.22410572454056, 32.94260371968307]"


In [129]:
geo_to_coords(fsub)

fsub

,SwisNo,AcceptedWa,County,cap_m3,geometry,coord
2,30-AB-0378,Green Materials,Orange,7645.5500,POINT (-117.7249 33.60377),"[-117.7249, 33.603770000000004]"
3,19-AR-5584,Green Materials,Los Angeles,33823.9132,POINT (-118.27222 34.11722),"[-118.27221999999999, 34.11722]"
6,37-AB-0011,Green Materials,San Diego,7645.5500,POINT (-117.1805 32.8622),"[-117.1805, 32.8622]"
10,13-AA-0095,"Agricultural,Manure",Imperial,54803.3024,POINT (-115.40676 32.97484),"[-115.40676, 32.97484]"


In [153]:
t = [[0,0],[0,1]]


C = list(csub.coord)
F = list(fsub.coord)

print("type C: ", type(C))
print("type t: ", type(t))

#  pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)
pd.DataFrame(distance_matrix(C, F), index = csub.COUNTY, columns = fsub.SwisNo)

# distance_matrix(C,F)

            

type C:  <class 'list'>
type t:  <class 'list'>


SwisNo,30-AB-0378,19-AR-5584,37-AB-0011,13-AA-0095
COUNTY,,,,
Los Angeles,1.006930,0.301419,1.894682,3.375724
Orange,0.149098,0.716878,0.952433,2.512408
Imperial,2.308586,3.014639,1.620438,0.234489
San Diego,0.829419,1.574251,0.091467,1.817632


In [138]:
from sklearn.metrics.pairwise import paired_distances
C = np.asarray(csub.coord)
F = np.asarray(fsub.coord)
paired_distances(C,F)

ValueError: setting an array element with a sequence.

In [ ]:
# calc euclidian distance between them --> get Dij 
# (multiply by 1.4 for 'real' distance, and again by '1.6' to proxy Lij)

# for i in range(len(csub.index)):
#     print(csub.geometry[i])
#     for j in range(len(fsub.index)):
#         print(j)

x = range(1,5)
y = range(6,10)

# arr = [[]]

# for i in : 
#     arr[]
D = csub.distance(fsub)
D

# arr == [[]]

# print('fsub', fsub)
# print(csub)

# print(fsub)
    
# {'col1': [1, 2], 'col2': [3, 4]}
test = {}

# for csubElem in enumerate(csub):
count = 0
for key, value in csub.items():    
    #print('*** KEY', key)
    #print('&&& VALUE', value)
    currentCount = 'i' + str(count)
    test[currentCount] = []
    count += 1
    for key1, value1 in fsub.items():
        point = csub.geometry
#         point1 = fsub.geometry
#         foo = point.distance(point1)
        #print("%% FOO")
        test[currentCount].append('point(s)')
        
# #     print(csub.geometry[i])
#     for j in enumerate(fsub):
#         print(i, j)
df = pd.DataFrame(data = test).T #this will transpose it
# df